# On Policy First Visit Monte Carlo Control
following Sutton barton book page 92


<img src="https://i.stack.imgur.com/033M8.png" 
     alt="Pseudo code" 
     style="width:700px;height:400px;"/>


In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
from collections import defaultdict
import sys

In [11]:
# First-visit MC prediction
def OnPolicyFirstVisitMCControl(eps=0.5):
    '''
    Written based on Sutton and Barto algorithm
    '''
    env = gym.make("CartPole-v1")
    buckets=(1, 1, 6, 12)
    nA = 2
    max_episodes = 20000
    gamma = 1    # don't want rewards to get smaller and smaller. We don't want to limit the time
    
    def discretize(obs):
        upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)]
        lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)]
        ratios = [(obs[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(obs))]
        new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(obs))]
        new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(obs))]
        return tuple(new_obs)
    
    def generateEpisode(policy, max_iterations):
        '''Generates an episode
        Ex. [(State0, Action0, Reward1), (State1, Action1, Reward2)]
        '''
        state = env.reset()
        episode = []
        for t in range(max_iterations):
            state = discretize(state)
            action = np.random.choice(2, 1, policy[state].tolist())[0]
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state
        return episode
            
    state_values = np.zeros(buckets)
    # arbitrary policy initialized with shape of the states. Only two actions. Need to sum the probability matrix to 1 tho
    policy_shape = buckets + (nA,)
#     policy = np.random.rand(*policy_shape)
    policy = np.full(policy_shape, 1/nA)
    # Q_values are just the average of the returns after many episodes
    Q_values = np.zeros(buckets + (nA,))
    Returns = defaultdict(list)
    
    for i in range(max_episodes):
        episode = generateEpisode(policy, 200)
        states_in_episode = list(set([sar for sar in episode]))
        # Expected sum of returns
        G = 0
        for state, action, reward in states_in_episode[::-1]:
            G = gamma * G + reward
            Returns[(state, action)].append(G)
            Q_values[state][action] = np.average(Returns[(state, action)])
            optimal_action = np.argmax(Q_values[state])

            # epsilon-soft policy
            for a in range(nA):
                if a == optimal_action:
                    policy[state][a] = 1 - eps + eps/nA
                else:
                    policy[state][a] = eps/nA
    return policy

In [12]:
def run_episode(env, policy, gamma = 1.0, render = True):
    """ Runs an episode and return the total reward """
    buckets=(1, 1, 6, 12)
    def discretize(obs):
        upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)]
        lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)]
        ratios = [(obs[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(obs))]
        new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(obs))]
        new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(obs))]
        return tuple(new_obs)
    
    state = env.reset()
    total_reward = 0
    max_iterations = 200
    
    for step_idx in range(max_iterations):
        if render and step_idx % 10 == 0:
            env.render()
        state = discretize(state)
        a = np.random.choice(2, 1, policy[state].tolist())[0]
        state, reward, done , _ = env.step(int(a))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
            
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    scores = [run_episode(env, policy, gamma, True) for _ in range(n)]
    return np.mean(scores)

In [13]:
optimal_policy = OnPolicyFirstVisitMCControl()

In [14]:
evaluate_policy(gym.make("CartPole-v1"), optimal_policy)

21.22

# Off Policy First Visit Monte Carlo Control

<img src="https://i.stack.imgur.com/Xi0vX.png" 
     alt="Pseudo code" 
     style="width:700px;height:400px;"/>


In [2]:
def OffPolicyMonteCarloControl(env, num_episodes, behavior_policy, gamma=1.0):
    '''
    Monte Carlo Control Off-Policy Control using Weight Importance Sampling
    Finds an optimal target greedy policy
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample
        behavior_policy: The behavior to follow while generating episodes.
            This is usually more exploratory
        discount_factor: Gamma discount factor
        
    Returns:
        A tuple (Q, policy)
        Q is a dictionary mapping state -> action values.
            policy is a function that takes an observation as an argument and returns
            action probabilities. This is the optimal greedy policy
    '''
    def discretize(obs):
        '''
            Discretizes the continuous state values to a discrete value
        '''
        buckets=(1, 1, 6, 12)    
        upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)]
        lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)]
        ratios = [(obs[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(obs))]
        new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(obs))]
        new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(obs))]
        return tuple(new_obs)
    
    def generateEpisode(policy, max_iterations):
        '''Generates an episode
        Ex. [(State0, Action0, Reward1), (State1, Action1, Reward2)]
        '''
        state = env.reset()
        episode = []
        for t in range(max_iterations):
            state = discretize(state)
            probs = behavior_policy(state)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state
        return episode
    
    def create_greedy_policy(Q):
        '''
        Creates a greedy policy based on Q values.

        Args:
            Q: A dictionary that maps from state -> action values

        Returns:
            A function that takes an observation as input and returns a vector
            of the highest possible action for each observation/state
        '''
        def policy_fn(state):
            A = np.zeros_like(Q[state], dtype=float)
            best_action = np.argmax(Q[state])   # Note: breaks tie with first one
            A[best_action] = 1.0
            return A
        return policy_fn

    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # the cumulative denominator of the weighted importance sampling formula
    # (across all episodes)
    C = defaultdict(lambda: np.zeros(env.action_space.n))
    target_policy = create_greedy_policy(Q)
    
    for i_episode in range(1, num_episodes + 1):
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        episode = generateEpisode(target_policy, 200)
        # Sum of discounted returns
        G = 0
        # Importance sampling ratio (the weights of the returns)
        W = 1.0
        
        for state, action, reward in episode[::-1]: # because the episodes are reversed, the returns in the end are weighted higher
            G = gamma * G + reward
            C[state][action] += W
            
            # This improves our target policy since it's a reference to Q
            Q[state][action] += (W / C[state][action]) * (G - Q[state][action])
            
            if action != np.argmax(target_policy(state)):
                break
            W = W * 1./behavior_policy(state)[action]   
            # Naturally this feels weird since the behavior policy is preset and you aren't updating the behavior policy
            # Also, here, if it were ordinary importance sampling, you'd be dividing by the number of time steps in which state s has visited
    return Q, target_policy

In [3]:
def create_random_policy(nA):
    """
    Creates a random policy function.
    
    Args:
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities
    """
    A = np.ones(nA, dtype=float) / nA
    def policy_fn(observation):
        return A
    return policy_fn

In [4]:
env = gym.make("CartPole-v1")
random_policy = create_random_policy(env.action_space.n)
Q, policy = OffPolicyMonteCarloControl(env, num_episodes=50000,behavior_policy=random_policy)

Episode 50000/50000.

In [15]:
def run_episode(env, policy, gamma = 1.0, render = True):
    """ Runs an episode and return the total reward """
    buckets=(1, 1, 6, 12)
    def discretize(obs):
        upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)]
        lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)]
        ratios = [(obs[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(obs))]
        new_obs = [int(round((buckets[i] - 1) * ratios[i])) for i in range(len(obs))]
        new_obs = [min(buckets[i] - 1, max(0, new_obs[i])) for i in range(len(obs))]
        return tuple(new_obs)
    
    state = env.reset()
    total_reward = 0
    max_iterations = 200
    
    for step_idx in range(max_iterations):
        if step_idx % 10 == 0:
            print(step_idx)
        if render:
            env.render()
        state = discretize(state)
        probs = policy(state)
        action = np.random.choice(np.arange(len(probs)), p=probs)
        state, reward, done, _ = env.step(action)
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    env.close()
    return total_reward

def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    scores = [run_episode(env, policy, gamma, True) for _ in range(n)]
    return np.mean(scores)

In [5]:
evaluate_policy(gym.make("CartPole-v1"), optimal_policy)

NameError: name 'evaluate_policy' is not defined